### Installation

In [1]:
%%capture
# We're installing the latest Torch, Triton, OpenAI's Triton kernels, Transformers and Unsloth!
!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers>=4.55.3" \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

### Unsloth

We're about to demonstrate the power of the new OpenAI GPT-OSS 20B model through a finetuning example. To use our `MXFP4` inference example, use this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/GPT_OSS_MXFP4_(20B)-Inference.ipynb) instead.

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit", # 20B model using bitsandbytes 4bit quantization
    "unsloth/gpt-oss-120b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b", # 20B model using MXFP4 format
    "unsloth/gpt-oss-120b",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = dtype, # None for auto detection
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.7: Fast Gpt_Oss patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.
Unsloth: Gpt_Oss does not support SDPA - switching to fast eager.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


In [22]:
import pandas as pd
df = pd.read_csv('/content/ticketMarche_test.csv')

In [19]:
df

,event_name,date,venue_name,Time,venue_coordinates,city,date_parsed
0,Youth Summit,19 & 20 Sep,American University in Cairo,3:00 PM,"30.019224469626735, 31.502954970733693",cairo,2025-09-19
1,Myriam Fares at KIKI’s White,Sep 19,"KIKI’s Beach, Hacienda White",09:00 PM,"31.002772740274153, 28.66518344233024",marsa matrouh,2025-09-19
2,Crowd Gigs By Mohamed Helmy - Saturday Early Show,Sep 20,Theatro Arkan,04:30 PM,"30.020402636742446, 31.003588270211274",giza,2025-09-20
3,Layali Marassi: Asala X Ahmed Saad,Sep 20,"Address Beach Resort, Marassi",11:00 PM,"30.966378514279903, 28.764663160486883",marsa matrouh,2025-09-20
4,Comedy Therapy,Sep 23,Cairo Jazz Club 610,09:00 PM,"30.019941913960334, 31.00298474601855",giza,2025-09-23
5,Cityscape Egypt 2025 - VIP Experience,24th to 27th of September 2025,Egypt International Exhibition Center,NaN,"30.0144858, 31.3842229",cairo,2025-09-24
6,20 years of Junior,Sep 25,Cairo Jazz Club 610,09:00 PM,"30.019941913960334, 31.00298474601855",giza,2025-09-25
7,Marwa Nagy - 50th anniversary of Umm Kalthoum,Sep 26,Theatro Arkan,08:00 PM,"30.020402636742446, 31.003588270211274",giza,2025-09-26
8,Cityscape Egypt 2025 - VIP Experience (Day 4),Sep 27,Egypt International Exhibition Center,12:00 PM,"30.0144858, 31.3842229",cairo,2025-09-27
9,SheArts,From 2nd to 5th Oct,Concert Hall (New Capital) - AUC Tahrir Square...,NaN,"30.042333968763398, 31.238579755550326",cairo,2025-10-02


In [90]:
import re

def predict_attendance(event):
    messages = [
      {
          "role": "user",
          "content": f"""
          Act as an event organizer that is trying to estimate
          how much people will attend an event based on the following events:
          Event: {event['event_name']}
          Venue: {event['venue_name']}
          Date: {event['date_parsed']}
          Time: {event['Time']}
          City: {event['city']}
        Take into consideration that the artist name is in the event name so you can use their popularity as an extra field for estiamtion.
        Respond ONLY with a single integer, inside <|final|> tags like this:
        THE OUTPUT MUST HAVE THE NUMBER IN THE FOLLOWING FORMAT <|final|>predicted number<|end|>
        Do NOT include <|analysis|>.
          """
      }
  ]

    # inputs = tokenizer(prompt, return_tensors="pt")
    inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high", # **NEW!** Set reasoning effort to low, medium or high
).to(model.device)

    gen_tokens = model.generate(
        **inputs,
        max_new_tokens=1024*2,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id

    )

    # keep only the new tokens (skip the prompt)
    new_tokens = gen_tokens[0][inputs["input_ids"].shape[-1]:]
    gen_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    match_ = re.search(r"<\|final\|>(\d+)(<\|end\|>)?", gen_text)
    event['estimated_attendance'] = int(match_.group(1))
    return (match_, gen_text, event)

In [96]:
match_, gen_text, event = predict_attendance(df.iloc[0,:])

In [97]:
gen_text

'analysisThe user wants a prediction of attendance of a youth summit at American University in Cairo, on 2025-09-19, at 3 PM. They mention "Take into consideration that the artist name is in the event name so you can use their popularity as an extra field for estimation." They didn\'t give artist name explicitly but the event is "Youth Summit" - no artist is mentioned. Is there an error? They mention "artist name is in the event name" so we can use their popularity as an extra field for estimation.\n\nWe need to produce a single integer predicted number. We must output <|final|>predicted number. No extra context, no analysis, no explanation.\n\nBut we need to produce a number. We need to somehow estimate the number of attendees.\n\nWe have limited data: It\'s a youth summit, presumably not huge, maybe some handful participants. Let\'s think.\n\nYouth Summit at American University in Cairo. Likely a conference for students, maybe 200-300 participants. The event might be a conference or 

In [98]:
match_.group(1)

'250'

In [99]:
event

,0
event_name,Youth Summit
date,19 & 20 Sep
venue_name,American University in Cairo
Time,3:00 PM
venue_coordinates,"30.019224469626735, 31.502954970733693"
city,cairo
date_parsed,2025-09-19
estimated_attendance,250
